In [22]:
import nbimporter
import pandas as pd
import numpy as np
import strategies_master
import inspect
import warnings
warnings.filterwarnings('ignore')
import importlib
importlib.reload(strategies_master)

<module 'strategies_master' from 'strategies_master.ipynb'>

In [2]:
raw_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw.csv.gz', compression='gzip')
raw_data['Date'] = pd.to_datetime(raw_data['Date'], format='mixed')
raw_data = raw_data.drop_duplicates()
list_of_stocks = raw_data['Stock Name'].unique().tolist()

In [3]:
def all_strategy_output_optimized(list_of_stocks):
    stock_data_dict = {stock: raw_data.loc[raw_data['Stock Name'] == stock] for stock in list_of_stocks}
    final_output = []
    
    for stock in list_of_stocks:
        data = stock_data_dict[stock]
        collector = []

        # Iterate over all strategy functions
        for name, func in inspect.getmembers(strategies_master, inspect.isfunction):
            if name.startswith("strategy"):
                result = func(data)
                result['strategy_name'] = name
                collector.append(result)

        # Concatenate all strategy results for the current stock
        if collector:
            collector_df = pd.concat(collector, ignore_index=True)
            final_output.append(collector_df)

    # Concatenate all stock results
    if final_output:
        final_output = pd.concat(final_output, ignore_index=True)
        
        # Replace -1 with 0 and fill NaNs
        final_output['Position'].replace(-1, 0, inplace=True)
        final_output = final_output.fillna(0)

        # Pivot the data to the desired format
        final_output = final_output.pivot_table(index=['Date', 'Stock Name'], columns='strategy_name', values='Position').reset_index()

    return final_output


In [7]:
backtester_input = all_strategy_output_optimized(list_of_stocks)

In [29]:
backtester_input = backtester_input.melt(id_vars=["Stock Name", "Date"], value_name= "Occurence", value_vars=backtester_input.set_index(["Date", "Stock Name"]).columns.tolist()).rename({"variable":"Strategy"}, axis = 1).query("Occurence == 1").drop(["Occurence"], axis = 1)

,Stock Name,Date,strategy_name
16073,20microns.NS,2020-01-15,strategy_10_ema_crossing
16075,360one.NS,2020-01-15,strategy_10_ema_crossing
16076,3iinfoltd.NS,2020-01-15,strategy_10_ema_crossing
16077,3mindia.NS,2020-01-15,strategy_10_ema_crossing
16078,3pland.NS,2020-01-15,strategy_10_ema_crossing
...,...,...,...
6391473,shripiston.NS,2024-08-19,strategy_20_ema_crossing_50_ema
6391480,siemens.NS,2024-08-19,strategy_20_ema_crossing_50_ema
6391671,titan.NS,2024-08-19,strategy_20_ema_crossing_50_ema
6391780,visakaind.NS,2024-08-19,strategy_20_ema_crossing_50_ema


In [8]:
backtester_input.to_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\backtester_input.csv.gz', index=False, compression='gzip')